In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
# load a pickle file
train_df = pd.read_pickle("../input/pickle-data/train_df.pkl")[:-1000]
test_df =  pd.read_pickle("../input/pickle-data/train_df.pkl")[-1000:]
final_test_df = pd.read_pickle("../input/pickle-data/test_df.pkl")

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize,stop_words='english', min_df=100)

# Create tfidf train and test variables
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['text']).toarray() 
y_train = train_df['emotion']
X_test_tfidf = tfidf_vectorizer.transform(test_df['text']).toarray() 
y_test = test_df['emotion']


In [ ]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder
import keras
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:]:\n', y_train[0:])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:]:\n', y_train[0:])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

In [ ]:
# I/O check
input_shape = X_train_tfidf.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=1024)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=1024)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

In [ ]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('/kaggle/working/training_log.csv')

# training setting
epochs = 10
batch_size = 128

# training!
history = model.fit(X_train_tfidf, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test_tfidf, y_test))
print('training finish')

In [ ]:
import numpy as np
pred_result = model.predict(X_test, batch_size=128)
pred_result[:5]
pred_result = label_decode(label_encoder, pred_result)

from sklearn.metrics import accuracy_score
print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 2)))

In [ ]:
final_X_test = tfidf_vectorizer.transform(final_test_df['text']).toarray() 
final_pred_result = model.predict(final_X_test, batch_size=128)
final_pred_result = label_decode(label_encoder, final_pred_result)
final_test_df['emotion'] = final_pred_result
csvData = final_test_df[['tweet_id','emotion']].to_csv('/kaggle/working/predict.csv',header=['id','emotion'],index=False) 